# Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from pathlib import Path
from dotenv import load_dotenv

from IPython.display import Markdown, display

import edurel.utils.dbcon as dbcu
import edurel.utils.db as dbu
import edurel.utils.duckdb as ddbu
import edurel.utils.llm as llmu
import edurel.utils.llmchat as llmc
import edurel.utils.yaml_utils as yu
import edurel.utils.misc as mu
import edurel.utils.mermaid as mmu

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases"


# Database

In [4]:
con = dbcu.adw_olap()
additional_fks = {
    "DimAccount": ["DimAccount|ParentAccountKey|AccountKey"],
    "DimDepartmentGroup": ["DimDepartmentGroup|ParentDepartmentGroupKey|DepartmentGroupKey"],
    "DimEmployee": ["DimEmployee|ParentEmployeeKey|EmployeeKey"],
    "DimOrganization": ["DimOrganization|ParentOrganizationKey|OrganizationKey"],
    }

db = dbu.DbHandler(con,additional_fks=additional_fks)
schema = db.schema_yaml_str(["nullable", "fkname"])

In [51]:
spec = """
del table pattern: DimAccount
del table pattern: DimDepartmentGroup
del table pattern: DimEmployee
del table pattern: DimOrganization
del table pattern: DimReseller
del table pattern: DimScenario
del table pattern: DimSalesReason
del table pattern: FactCurrencyRate
del table pattern: FactAdditionalInternationalProductDescription
del table pattern: FactCallCenter
del table pattern: FactFinance
del table pattern: FactInternetSalesReason
del table pattern: FactProductInventory
del table pattern: FactResellerSales
del table pattern: FactSalesQuota
del table pattern: FactSurveyResponse
del table pattern: NewFactCurrencyRate
del table pattern: ProspectiveBuyer
del column FactInternetSales index: [8:17, 19:]
del column DimPromotion index: [1, 3, 4, 7, 8, 10, 11]
del column DimProduct index: [1, 3, 4, 6, 7, 8:]
del column DimProductSubcategory index: [1, 3, 4]
del column DimProductCategory index: [1, 3, 4]
del column DimCustomer index: [2, 3, 5, 7:]
del column DimGeography index: [6, 7, 10]

"""
reduced_schema = yu.yaml_to_yaml(schema, spec)
print(reduced_schema)

tables:
- tablename: DimCurrency
  columns:
  - columnname: CurrencyKey
    type: INTEGER
  - columnname: CurrencyAlternateKey
    type: VARCHAR
  - columnname: CurrencyName
    type: VARCHAR
  primary_key:
  - CurrencyKey
- tablename: DimCustomer
  columns:
  - columnname: CustomerKey
    type: INTEGER
  - columnname: GeographyKey
    type: INTEGER
  - columnname: FirstName
    type: VARCHAR
  - columnname: LastName
    type: VARCHAR
  primary_key:
  - CustomerKey
  foreign_keys:
  - sourcecolumns:
    - GeographyKey
    targettable: DimGeography
    targetcolumns:
    - GeographyKey
- tablename: DimDate
  columns:
  - columnname: DateKey
    type: INTEGER
  - columnname: FullDateAlternateKey
    type: DATE
  - columnname: DayNumberOfWeek
    type: INTEGER
  - columnname: EnglishDayNameOfWeek
    type: VARCHAR
  - columnname: SpanishDayNameOfWeek
    type: VARCHAR
  - columnname: FrenchDayNameOfWeek
    type: VARCHAR
  - columnname: DayNumberOfMonth
    type: INTEGER
  - columnname: D

In [52]:
mmu.schema_mermaid_png(reduced_schema, output_path="schema.png", omit_tags=["nullable", "fkname"], direction="LR", scale=6)